In [ ]:
import numpy as np 
import pandas as pd
import os

In [ ]:
petdata=pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
petdata.head()

In [ ]:
from tensorflow.keras.applications import ResNet50, InceptionV3, InceptionResNetV2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import preprocess_input

#resnet_weights_path='../input/keras-pretrained-models/resnet50_weights_tf_dim_ordering_tf_kernels.h5'
base_model = ResNet50(weights='../input/tf-keras-pretrained-model-weights/Top/resnet50_weights_tf_dim_ordering_tf_kernels.h5')
model = Model(inputs=base_model.input, outputs=base_model.get_layer('avg_pool').output)
features_array=np.zeros((9912,2048))

image_size = 224
count=0

for i,file in enumerate(petdata['Id']):
    img = image.load_img(os.path.join('../input/petfinder-pawpularity-score/train',file+'.jpg'), target_size=(image_size, image_size))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    features = model.predict(x)
    features = features.reshape(2048,)
    features_array[i,:] = features

    if(count%1000==0):
        print(count)
    count+=1

In [ ]:
print(features_array.shape)

smallpetdata = pd.DataFrame(features_array)
smallpetdata = pd.concat([petdata,smallpetdata], axis=1, join='inner')
smallpetdata.head()

In [ ]:
from sklearn.model_selection import train_test_split

train_set,val_set = train_test_split(smallpetdata,test_size=0.2,random_state=42)

In [ ]:
print(train_set.shape,val_set.shape)

X_train=train_set.drop(['Id','Pawpularity'],axis=1)
y_train=train_set['Pawpularity']

X_val=val_set.drop(['Id','Pawpularity'],axis=1)
y_val=val_set['Pawpularity']

In [ ]:
from sklearn.decomposition import PCA

pca=PCA(n_components=0.9)
X_reduced=pca.fit_transform(X_train)


In [ ]:
X_val_reduced=pca.transform(X_val)

In [ ]:
X_reduced.shape

In [ ]:
from sklearn.linear_model import LinearRegression
lin_reg=LinearRegression()
lin_reg.fit(X_reduced,y_train)

In [ ]:
from sklearn.metrics import mean_squared_error

y_predicted=lin_reg.predict(X_val_reduced)
np.sqrt(mean_squared_error(y_val,y_predicted))

In [ ]:
'''from sklearn.ensemble import RandomForestRegressor
rf_reg=RandomForestRegressor(random_state=0,max_depth=2)
rf_reg.fit(X_train, y_train)'''

In [ ]:
'''from sklearn.metrics import mean_squared_error

y_predicted=rf_reg.predict(X_val)
np.sqrt(mean_squared_error(y_val,y_predicted))'''

In [ ]:
'''from sklearn.svm import SVR
svm_reg=SVR(kernel='rbf')
svm_reg.fit(X_train,y_train)'''

In [ ]:
'''from sklearn.metrics import mean_squared_error

y_predicted=svm_reg.predict(X_val)
np.sqrt(mean_squared_error(y_val,y_predicted))'''

In [ ]:
test_data=pd.read_csv('../input/petfinder-pawpularity-score/test.csv')
test_data.head()

In [ ]:
image_size = 224
features_array=np.zeros((test_data.shape[0],2048))

for i,file in enumerate(test_data['Id']):
    img = image.load_img(os.path.join('../input/petfinder-pawpularity-score/test',file+'.jpg'), target_size=(image_size, image_size))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    
    features = model.predict(x)
    features = features.reshape(2048,)
    features_array[i,:] = features

In [ ]:
testpetdata = pd.DataFrame(features_array)
testpetdata = pd.concat([test_data,testpetdata], axis=1, join='inner')
testpetdata.head()

In [ ]:
X_test=testpetdata.drop('Id',axis=1)
X_reduced_test=pca.transform(X_test)

y_test=lin_reg.predict(X_reduced_test)
X_test.head()

In [ ]:
out={}
out['Id']=testpetdata.Id
out['Pawpularity']=y_test
out=pd.DataFrame(out)

In [ ]:
out.to_csv('submission.csv',index=False)